Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size], stddev=math.sqrt(2./(image_size*image_size))))
  biases1 = tf.Variable(tf.zeros([hidden_size]))

  weights2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels], stddev=math.sqrt(2./hidden_size)))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  pre_act = tf.matmul(tf_train_dataset, weights1) + biases1
  post_act = tf.nn.relu(pre_act)
  
  logits = tf.matmul(post_act, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))


  regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) +
                  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
  loss += 1e-4 * regularizers

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2)+biases2
    )
  test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2)+biases2
    )

In [6]:
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 3000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.511641
Minibatch accuracy: 5.5%
Validation accuracy: 32.6%
Minibatch loss at step 3000: 0.519027
Minibatch accuracy: 86.7%
Validation accuracy: 88.9%
Minibatch loss at step 6000: 0.505567
Minibatch accuracy: 89.1%
Validation accuracy: 89.1%
Minibatch loss at step 9000: 0.351257
Minibatch accuracy: 90.6%
Validation accuracy: 89.4%
Minibatch loss at step 12000: 0.232053
Minibatch accuracy: 96.1%
Validation accuracy: 89.3%
Minibatch loss at step 15000: 0.332197
Minibatch accuracy: 93.0%
Validation accuracy: 89.4%
Minibatch loss at step 18000: 0.319506
Minibatch accuracy: 93.0%
Validation accuracy: 87.6%
Minibatch loss at step 21000: 0.418762
Minibatch accuracy: 94.5%
Validation accuracy: 88.1%
Minibatch loss at step 24000: 0.248118
Minibatch accuracy: 97.7%
Validation accuracy: 89.4%
Minibatch loss at step 27000: 0.237264
Minibatch accuracy: 96.1%
Validation accuracy: 89.7%
Minibatch loss at step 30000: 0.407941
Minibatch accuracy: 95.3%
Validation 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
train_dataset_extreme = train_dataset[:256]
train_labels_extreme = train_labels[:256]

num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_extreme.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_extreme[offset:(offset + batch_size), :]
    batch_labels = train_labels_extreme[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 3000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.544614
Minibatch accuracy: 7.8%
Validation accuracy: 31.2%
Minibatch loss at step 3000: 0.062067
Minibatch accuracy: 100.0%
Validation accuracy: 70.9%
Minibatch loss at step 6000: 0.046916
Minibatch accuracy: 100.0%
Validation accuracy: 71.1%
Minibatch loss at step 9000: 0.035704
Minibatch accuracy: 100.0%
Validation accuracy: 71.1%
Minibatch loss at step 12000: 0.027402
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 15000: 0.021254
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 18000: 0.016697
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Minibatch loss at step 21000: 0.013317
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 24000: 0.010810
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 27000: 0.008948
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 30000: 0.007564
Minibatch accuracy: 100.0%
V

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size], stddev=math.sqrt(2./(image_size*image_size))))
  biases1 = tf.Variable(tf.zeros([hidden_size]))

  weights2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels], stddev=math.sqrt(2./hidden_size)))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  tf_train_dataset = tf.nn.dropout(tf_train_dataset, 0.8)
  pre_act = tf.matmul(tf_train_dataset, weights1) + biases1
  post_act = tf.nn.relu(pre_act)
  dropout_layer1 = tf.nn.dropout(post_act, 0.6)

  logits = tf.matmul(dropout_layer1, weights2) + biases2
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))


  regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) +
                  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
  loss += 1e-4 * regularizers

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2)+biases2
    )
  test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2)+biases2
    )

In [9]:
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 3000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.674128
Minibatch accuracy: 9.4%
Validation accuracy: 41.1%
Minibatch loss at step 3000: 0.581215
Minibatch accuracy: 86.7%
Validation accuracy: 88.3%
Minibatch loss at step 6000: 0.534361
Minibatch accuracy: 89.8%
Validation accuracy: 89.3%
Minibatch loss at step 9000: 0.407498
Minibatch accuracy: 89.8%
Validation accuracy: 89.3%
Minibatch loss at step 12000: 0.400858
Minibatch accuracy: 89.8%
Validation accuracy: 89.1%
Minibatch loss at step 15000: 0.432319
Minibatch accuracy: 89.8%
Validation accuracy: 89.8%
Minibatch loss at step 18000: 0.451471
Minibatch accuracy: 89.8%
Validation accuracy: 89.3%
Minibatch loss at step 21000: 0.444680
Minibatch accuracy: 93.8%
Validation accuracy: 89.9%
Minibatch loss at step 24000: 0.348698
Minibatch accuracy: 93.0%
Validation accuracy: 89.9%
Minibatch loss at step 27000: 0.355794
Minibatch accuracy: 89.8%
Validation accuracy: 90.0%
Minibatch loss at step 30000: 0.532523
Minibatch accuracy: 89.1%
Validation 

In [10]:
train_dataset_extreme = train_dataset[:256]
train_labels_extreme = train_labels[:256]

num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_extreme.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_extreme[offset:(offset + batch_size), :]
    batch_labels = train_labels_extreme[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 3000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.691259
Minibatch accuracy: 7.8%
Validation accuracy: 35.3%
Minibatch loss at step 3000: 0.062373
Minibatch accuracy: 100.0%
Validation accuracy: 71.6%
Minibatch loss at step 6000: 0.047189
Minibatch accuracy: 100.0%
Validation accuracy: 71.8%
Minibatch loss at step 9000: 0.035961
Minibatch accuracy: 100.0%
Validation accuracy: 71.8%
Minibatch loss at step 12000: 0.027725
Minibatch accuracy: 100.0%
Validation accuracy: 71.8%
Minibatch loss at step 15000: 0.021501
Minibatch accuracy: 100.0%
Validation accuracy: 71.9%
Minibatch loss at step 18000: 0.016852
Minibatch accuracy: 100.0%
Validation accuracy: 71.8%
Minibatch loss at step 21000: 0.013616
Minibatch accuracy: 100.0%
Validation accuracy: 71.7%
Minibatch loss at step 24000: 0.011071
Minibatch accuracy: 100.0%
Validation accuracy: 71.6%
Minibatch loss at step 27000: 0.009109
Minibatch accuracy: 100.0%
Validation accuracy: 71.6%
Minibatch loss at step 30000: 0.007803
Minibatch accuracy: 100.0%
V

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [13]:
batch_size = 128
hidden_size1 = 1024
hidden_size2 = 512
hidden_size3 = 256
hidden_size4 = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size1], stddev=math.sqrt(2./(image_size*image_size))))
  biases1 = tf.Variable(tf.zeros([hidden_size1]))

  weights2 = tf.Variable(
    tf.truncated_normal([hidden_size1, hidden_size2], stddev=math.sqrt(2./hidden_size1)))
  biases2 = tf.Variable(tf.zeros([hidden_size2]))

  weights3 = tf.Variable(
    tf.truncated_normal([hidden_size2, hidden_size3], stddev=math.sqrt(2./hidden_size2)))
  biases3 = tf.Variable(tf.zeros([hidden_size3]))

  weights4 = tf.Variable(
    tf.truncated_normal([hidden_size3, hidden_size4], stddev=math.sqrt(2./hidden_size3)))
  biases4 = tf.Variable(tf.zeros([hidden_size4]))
    
  weights5 = tf.Variable(
    tf.truncated_normal([hidden_size4, num_labels], stddev=math.sqrt(2./hidden_size4)))
  biases5 = tf.Variable(tf.zeros([num_labels]))
  
  
  # Training computation.
  tf_train_dataset = tf.nn.dropout(tf_train_dataset, 0.8)
  pre_act1 = tf.matmul(tf_train_dataset, weights1) + biases1
  post_act1 = tf.nn.relu(pre_act1)
  dropout_layer1 = tf.nn.dropout(post_act1, 0.8)

  pre_act2 = tf.matmul(dropout_layer1, weights2) + biases2
  post_act2 = tf.nn.relu(pre_act2)
  dropout_layer2 = tf.nn.dropout(post_act2, 0.8)

  pre_act3 = tf.matmul(dropout_layer2, weights3) + biases3
  post_act3 = tf.nn.relu(pre_act3)
  dropout_layer3 = tf.nn.dropout(post_act3, 0.8)

  pre_act4 = tf.matmul(dropout_layer3, weights4) + biases4
  post_act4 = tf.nn.relu(pre_act4)
  dropout_layer4 = tf.nn.dropout(post_act4, 0.8)
    
  logits = tf.matmul(dropout_layer4, weights5) + biases5

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))


  regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) +
                  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2) +
                  tf.nn.l2_loss(weights3) + tf.nn.l2_loss(biases3) +
                  tf.nn.l2_loss(weights4) + tf.nn.l2_loss(biases4) +
                  tf.nn.l2_loss(weights5) + tf.nn.l2_loss(biases5) )
  loss += 1e-4 * regularizers
  
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)


  valid_pre_act1 = tf.matmul(tf_valid_dataset, weights1) + biases1
  valid_post_act1 = tf.nn.relu(valid_pre_act1)
  valid_pre_act2 = tf.matmul(valid_post_act1, weights2) + biases2
  valid_post_act2 = tf.nn.relu(valid_pre_act2)
  valid_pre_act3 = tf.matmul(valid_post_act2, weights3) + biases3
  valid_post_act3 = tf.nn.relu(valid_pre_act3)  
  valid_pre_act4 = tf.matmul(valid_post_act3, weights4) + biases4
  valid_post_act4 = tf.nn.relu(valid_pre_act4)  
  valid_prediction = tf.nn.softmax( tf.matmul(valid_post_act4, weights5) + biases5 )
    
  test_pre_act1 = tf.matmul(tf_test_dataset, weights1) + biases1
  test_post_act1 = tf.nn.relu(test_pre_act1)
  test_pre_act2 = tf.matmul(test_post_act1, weights2) + biases2
  test_post_act2 = tf.nn.relu(test_pre_act2)
  test_pre_act3 = tf.matmul(test_post_act2, weights3) + biases3
  test_post_act3 = tf.nn.relu(test_pre_act3)   
  test_pre_act4 = tf.matmul(test_post_act3, weights4) + biases4
  test_post_act4 = tf.nn.relu(test_pre_act4)     
  test_prediction = tf.nn.softmax( tf.matmul(test_post_act4, weights5) + biases5 )
    

In [14]:
num_steps = 140001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 14000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.631862
Minibatch accuracy: 10.9%
Validation accuracy: 10.3%
Minibatch loss at step 14000: 0.861485
Minibatch accuracy: 75.8%
Validation accuracy: 87.0%
Minibatch loss at step 28000: 0.533287
Minibatch accuracy: 88.3%
Validation accuracy: 89.0%
Minibatch loss at step 42000: 0.511270
Minibatch accuracy: 89.1%
Validation accuracy: 89.9%
Minibatch loss at step 56000: 0.407863
Minibatch accuracy: 93.8%
Validation accuracy: 90.4%
Minibatch loss at step 70000: 0.534386
Minibatch accuracy: 88.3%
Validation accuracy: 90.9%
Minibatch loss at step 84000: 0.358785
Minibatch accuracy: 90.6%
Validation accuracy: 91.1%
Minibatch loss at step 98000: 0.359739
Minibatch accuracy: 94.5%
Validation accuracy: 91.3%
Minibatch loss at step 112000: 0.290494
Minibatch accuracy: 94.5%
Validation accuracy: 91.3%
Minibatch loss at step 126000: 0.330977
Minibatch accuracy: 93.8%
Validation accuracy: 91.6%
Minibatch loss at step 140000: 0.315237
Minibatch accuracy: 94.5%
Vali